In [1]:
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
sys.path.append('../code/')
%matplotlib inline
import itertools
from pathlib import Path
from torch.autograd import Variable


In [36]:
valid_error = {}
dataset =3
gid = 0
for fold_num in range(5):
    valid_error[fold_num] = {}
    for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
        valid_error[fold_num][appliance] = {}
        for lr in [0.001, 0.01, 0.1]:
            valid_error[fold_num][appliance][lr] = {}
            for iters in [200000]:
            
                directory = "../code/baseline/cnn-tree/{}/{}/{}/{}/0.0/".format(dataset, fold_num, lr, iters)
                filename = "valid-error-[\'{}\'].npy".format(appliance)
                
                full_path = directory + filename
                my_file = Path(full_path)
                if not my_file.exists():
                    print("CUDA_VISIBLE_DEVICES={} python cnn-tree.py {} {} {} 0 {} {} &".format(gid, dataset, lr, iters, fold_num, appliance))
                    gid += 1
                    if gid == 4:
                        gid = 0
                else:
                    for it in range(1000, 200000+1, 1000):
                        valid_error[fold_num][appliance][lr][it] = np.load(full_path).item()[it][appliance]

In [37]:
best_param = {}
for fold_num in range(5):
    best_param[fold_num] = {}

for fold_num in range(5):
    error = np.inf
    for lr in [0.001, 0.01, 0.1]:
        for iters in range(1000, 200000+1, 1000):
            cur_error = 0
            for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
                cur_error += valid_error[fold_num][appliance][lr][iters]
            if cur_error < error:
                error = cur_error
                best_param[fold_num]['lr'] = lr
                best_param[fold_num]['iters'] = iters

In [38]:
best_error = {}
for fold_num in range(5):
    best_error[fold_num] = {}
    lr = best_param[fold_num]['lr']
    iters = best_param[fold_num]['iters']
    
    directory = "../code/baseline/cnn-tree/{}/{}/{}/200000/0.0/".format(dataset, fold_num, lr)
    for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
        filename = "test-error-[\'{}\'].npy".format(appliance)
        full_path = directory + filename
        best_error[fold_num][appliance] = np.load(full_path).item()[iters][appliance]


In [39]:
def get_overall_error(best_error):
    error = {}
    for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
        error[appliance] = best_error[0][appliance]*14 + best_error[1][appliance]*14 + best_error[2][appliance]*14 + best_error[3][appliance]*13 + best_error[4][appliance]*13
        print(error[appliance])
        error[appliance] /= 68
    return error

In [40]:
get_overall_error(best_error)

21150.2033418
2242.25787028
4681.70005103
1065.1152991
582.391969536


{'dr': 68.848530162278607,
 'dw': 15.663460280944445,
 'fridge': 32.974380445304796,
 'hvac': 311.03240208594184,
 'mw': 8.5645877872962792}

In [41]:
np.save("../code/baseline/result/cnn-individual-param-{}.npy".format(dataset), best_param)